#### Jasmine Malik (jm5xx@virginia.edu) DS 5001 Spring 2023

#### Creating Corpus

In [24]:
import pandas as pd
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
#from textimporter import TextImporter
#from langmod import NgramCounter, NgramLanguageModel

In [25]:
#from textparserjm1 import TextParser

In [26]:
print(pd.__version__)

1.5.3


In [27]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

In [28]:
source_files = '100_english_novels-master/corpus/project'

In [29]:
source_file_list = sorted(glob(f"{source_files}/*.*"))

In [30]:
book_data = []
book_id_count = 1
for source_file_path in source_file_list:
    book_id = book_id_count
    book_title = source_file_path.split('/')[3].split('_')[1]
    book_data.append((book_id, source_file_path, book_title))
    book_id_count += 1

In [31]:
LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()

In [32]:
LIB.head()

,source_file_path,raw_title
book_id,,
1,100_english_novels-master/corpus/project/Barcl...,Ladies
2,100_english_novels-master/corpus/project/Barcl...,Postern
3,100_english_novels-master/corpus/project/Benne...,Babylon
4,100_english_novels-master/corpus/project/Benne...,Helen
5,100_english_novels-master/corpus/project/Burne...,Garden


In [33]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
clip_pats = [
    r"",
    r""
]
# All are 'chap'and 'm'
ohco_pat_list = [
    (1,  rf"CHAPTER\s[A-Za-z]"),
    (2,   rf"^THE\s[A-Za-z]+ DAY$"),
    (3,   rf"^Chapter\s"),
    (4, rf"^Chapter\s[0-9]+"),
    (5, rf"^CHAPTER\s[A-Za-z]"),
    (6, rf"^\d\s"),
    (7, rf"^\s[A-Za-z]+ "),
    (8,  rf"^CHAPTER\s[A-Za-z]"),
    (9, rf""),
    (10, rf"^CHAPTER [A-Za-z]+"),
    (11, rf"^Chapter\s[A-Za-z]"),
    (12, rf"^\sCHAPTER\s[A-Za-z]"),
    (13, rf"^CHAPTER\s[A-Za-z]"),
    (14, rf"^CHAPTER\s[A-Za-z]+\."),
    (15, rf"^CHAPTER\s[A-Za-z]+\."),
    (16, rf"^[IVXLCM]+"),
    (17, rf"^[IVXLCM]+"),
    (18, rf"^[IVXLCM]+"),
    (19, rf"^[0-9]+"),
    (20, rf"^CHAPTER\s[IVXLCM]+")
]

In [34]:
LIB['regex'] = ohco_pat_list

In [35]:
LIB['regex'] = LIB['regex'].astype(str)

In [36]:
LIB['regex'] = LIB['regex'].str.split(',',expand=True)[1].str[:-1]

In [37]:
LIB

,source_file_path,raw_title,regex
book_id,,,
1,100_english_novels-master/corpus/project/Barcl...,Ladies,'CHAPTER\\s[A-Za-z]'
2,100_english_novels-master/corpus/project/Barcl...,Postern,'^THE\\s[A-Za-z]+ DAY$'
3,100_english_novels-master/corpus/project/Benne...,Babylon,'^Chapter\\s'
4,100_english_novels-master/corpus/project/Benne...,Helen,'^Chapter\\s[0-9]+'
5,100_english_novels-master/corpus/project/Burne...,Garden,'^CHAPTER\\s[A-Za-z]'
6,100_english_novels-master/corpus/project/Burne...,Princess,'^\\d\\s'
7,100_english_novels-master/corpus/project/Chest...,Innocence,'^\\s[A-Za-z]+ '
8,100_english_novels-master/corpus/project/Chest...,Napoleon,'^CHAPTER\\s[A-Za-z]'
9,100_english_novels-master/corpus/project/Conra...,Almayer,''


### Make the first dataframe

In [42]:
text_file = LIB['source_file_path'].iloc[0]
text_file

'100_english_novels-master/corpus/project/Barclay_Ladies_1917.txt'

In [43]:
text_file = LIB['source_file_path'].iloc[0]
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), 
    columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()

title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z]", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()

The White Ladies of Worcester


<ipython-input-43-c737dd984a06>:17: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


In [48]:
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack().to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()

In [49]:
TOKENS

pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                               
1        0        0        0         (The, DT)   DT        The        the
                           1   (slanting, VBG)  VBG   slanting   slanting
                           2       (rays, NNS)  NNS       rays       rays
                           3          (of, IN)   IN         of         of
                           4   (afternoon, NN)   NN  afternoon  afternoon
...                                        ...  ...        ...        ...
60       74       0        19         (to, TO)   TO         to         to
                           20        (the, DT)   DT        the        the
                           21    (evening, NN)   NN    evening    evening
                           22       (star, NN)   NN       star       star
         75       0        0         (END, NN)   NN        END        end

[121859 rows x 4 columns]

In [50]:
df1 = TOKENS
df1['book_id'] = 1

### DF2

In [52]:
text_file = LIB['source_file_path'].iloc[1]
text_file

'100_english_novels-master/corpus/project/Barclay_Postern_1911.txt'

In [53]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
LINES.iloc[41:60]
chap_lines = LINES.line_str.str.match(r"THE\s[A-Za-z]+\sDAY", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

Through the Postern Gate


<ipython-input-53-8e5ac29f8bfc>:15: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                              
6        0        0        0         (AN, DT)   DT         AN         an
                           1  (INTERLUDE, NN)   NN  INTERLUDE  interlude
         1        1        0         (AS, IN)   IN         AS         as
                           1         (A, NNP)  NNP          A          a
                           2    (DREAM,, NNP)  NNP     DREAM,     dream,
...                                       ...  ...        ...        ...
14       149      1        6      (were, VBD)  VBD       were       were
                           7        (the, DT)   DT        the        the
                           8    (seventh, JJ)   JJ    seventh    seventh
                           9        (day, NN)   NN        day        day
         150      0        0        (END, NN)   NN        END        end

[39870 rows x 4 columns]

In [54]:
df2 = TOKENS
df2['book_id'] = 2
CORPUS = pd.concat([df1,df2], keys=[1,2], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                      
1       1        0        0        0        (The, DT)   DT        The   
                                   1  (slanting, VBG)  VBG   slanting   
                                   2      (rays, NNS)  NNS       rays   
                                   3         (of, IN)   IN         of   
                                   4  (afternoon, NN)   NN  afternoon   
...                                               ...  ...        ...   
2       14       149      1        6      (were, VBD)  VBD       were   
                                   7        (the, DT)   DT        the   
                                   8    (seventh, JJ)   JJ    seventh   
                                   9        (day, NN)   NN        day   
                 150      0        0        (END, NN)   NN        END   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
2       14       149      1        6       were        2  
                                   7        the        2  
                                   8    seventh        2  
                                   9        day        2  
                 150      0        0        end        2  

[161729 rows x 5 columns]

### DF3

In [55]:
text_file = LIB['source_file_path'].iloc[2]
text_file

'100_english_novels-master/corpus/project/Bennet_Babylon_1902.txt'

In [56]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
LINES.iloc[41:60]
chap_lines = LINES.line_str.str.match(r"Chapter\s[A-Za-z]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

THE GRAND BABYLON H"TEL


<ipython-input-56-5cf70f1720f1>:15: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos token_str term_str
chap_num para_num sent_num                                          
1        0        0        0     ('YES,, CD)   CD     'YES,    'yes,
                           1       (sir, NN)   NN       sir      sir
                  1        0         (', '')   ''         '        '
         1        0        0   (Jules,, NNP)  NNP    Jules,   jules,
                           1       (the, DT)   DT       the      the
...                                      ...  ...       ...      ...
30       37       1        2       (the, DT)   DT       the      the
                           3    (Grand, NNP)  NNP     Grand    grand
                           4  (Babylon, NNP)  NNP   Babylon  babylon
                           5        (H, NNP)  NNP         H        h
                  2        0       (tel, NN)   NN       tel      tel

[65965 rows x 4 columns]

In [57]:
df3 = TOKENS
df3['book_id'] = 3
CORPUS = pd.concat([df1,df2,df3], keys=[1,2,3], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                      
1       1        0        0        0        (The, DT)   DT        The   
                                   1  (slanting, VBG)  VBG   slanting   
                                   2      (rays, NNS)  NNS       rays   
                                   3         (of, IN)   IN         of   
                                   4  (afternoon, NN)   NN  afternoon   
...                                               ...  ...        ...   
3       30       37       1        2        (the, DT)   DT        the   
                                   3     (Grand, NNP)  NNP      Grand   
                                   4   (Babylon, NNP)  NNP    Babylon   
                                   5         (H, NNP)  NNP          H   
                          2        0        (tel, NN)   NN        tel   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
3       30       37       1        2        the        3  
                                   3      grand        3  
                                   4    babylon        3  
                                   5          h        3  
                          2        0        tel        3  

[227694 rows x 5 columns]

### DF4

In [58]:
text_file = LIB['source_file_path'].iloc[3]
text_file

'100_english_novels-master/corpus/project/Bennet_Helen_1910.txt'

In [59]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"Chapter\s[0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

Helen with the high hand


<ipython-input-59-3f850bab4c35>:14: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos token_str term_str
chap_num para_num sent_num                                         
1        0        0        0       (In, IN)   IN        In       in
                           1      (the, DT)   DT       the      the
                           2    (Five, NNP)  NNP      Five     five
                           3   (Towns, NNP)  NNP     Towns    towns
                           4    (human, JJ)   JJ     human    human
...                                     ...  ...       ...      ...
27       79       3        4  (better, RBR)  RBR    better   better
                           5     (than, IN)   IN      than     than
                           6     (ever, RB)   RB      ever     ever
         80       0        0      (THE, DT)   DT       THE      the
                           1      (END, NN)   NN       END      end

[52456 rows x 4 columns]

In [60]:
df4 = TOKENS
df4['book_id'] = 4
CORPUS = pd.concat([df1,df2,df3,df4], keys=[1,2,3,4], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                      
1       1        0        0        0        (The, DT)   DT        The   
                                   1  (slanting, VBG)  VBG   slanting   
                                   2      (rays, NNS)  NNS       rays   
                                   3         (of, IN)   IN         of   
                                   4  (afternoon, NN)   NN  afternoon   
...                                               ...  ...        ...   
4       27       79       3        4    (better, RBR)  RBR     better   
                                   5       (than, IN)   IN       than   
                                   6       (ever, RB)   RB       ever   
                 80       0        0        (THE, DT)   DT        THE   
                                   1        (END, NN)   NN        END   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
4       27       79       3        4     better        4  
                                   5       than        4  
                                   6       ever        4  
                 80       0        0        the        4  
                                   1        end        4  

[280150 rows x 5 columns]

### DF5

In [61]:
text_file = LIB['source_file_path'].iloc[4]
text_file

'100_english_novels-master/corpus/project/Burnett_Garden_1911.txt'

In [62]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

THE SECRET GARDEN


In [63]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-63-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                                
1        0        0        0        (When, WRB)  WRB       When       when
                           1        (Mary, NNP)  NNP       Mary       mary
                           2      (Lennox, NNP)  NNP     Lennox     lennox
                           3         (was, VBD)  VBD        was        was
                           4        (sent, VBN)  VBN       sent       sent
...                                         ...  ...        ...        ...
27       92       1        24         (boy, NN)   NN        boy        boy
                           25          (in, IN)   IN         in         in
                           26  (Yorkshire, NNP)  NNP  Yorkshire  yorkshire
                           27     (Master, NNP)  NNP     Master     master
                           28      (Colin, NNP)  NNP      Colin      colin

[80880 rows x 4 columns]

In [69]:
df5 = TOKENS
df5['book_id'] = 5
CORPUS = pd.concat([df1,df2,df3,df4,df5], keys=[1,2,3,4,5], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
5       19       44       0        39        (the, DT)   DT        the   
                                   40   (carriage, NN)   NN   carriage   
                                   41        (and, CC)   CC        and   
                                   42      (drove, VB)   VB      drove   
                                   43       (away, RB)   RB       away   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
5       19       44       0        39        the        5  
                                   40   carriage        5  
                                   41        and        5  
                                   42      drove        5  
                                   43       away        5  

[346948 rows x 5 columns]

In [70]:
CORPUS.to_csv('CORPUS6.csv')

### df6

In [71]:
text_file = LIB['source_file_path'].iloc[5]
text_file

'100_english_novels-master/corpus/project/Burnett_Princess_1905.txt'

In [72]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"[0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

A Little Princess


In [73]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-73-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple pos token_str  term_str
chap_num para_num sent_num                                           
1        0        0        0       (Once, RB)  RB      Once      once
                           1         (on, IN)  IN        on        on
                           2          (a, DT)  DT         a         a
                           3       (dark, JJ)  JJ      dark      dark
                           4   (winter's, NN)  NN  winter's  winter's
...                                       ...  ..       ...       ...
19       44       0        39       (the, DT)  DT       the       the
                           40  (carriage, NN)  NN  carriage  carriage
                           41       (and, CC)  CC       and       and
                           42     (drove, VB)  VB     drove     drove
                           43      (away, RB)  RB      away      away

[66798 rows x 4 columns]

In [74]:
df6 = TOKENS
df6['book_id'] = 6
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6], keys=[1,2,3,4,5,6], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
6       19       44       0        39        (the, DT)   DT        the   
                                   40   (carriage, NN)   NN   carriage   
                                   41        (and, CC)   CC        and   
                                   42      (drove, VB)   VB      drove   
                                   43       (away, RB)   RB       away   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
6       19       44       0        39        the        6  
                                   40   carriage        6  
                                   41        and        6  
                                   42      drove        6  
                                   43       away        6  

[413746 rows x 5 columns]

### df7

In [75]:
text_file = LIB['source_file_path'].iloc[6]
text_file

'100_english_novels-master/corpus/project/Chesterton_Innocence_1911.txt'

In [76]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[1].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"\s[A-Za-z0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

 THE INNOCENCE OF FATHER BROWN



In [79]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS = TOKENS.sort_index()
TOKENS

<ipython-input-79-7587d0b24ae6>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos token_str term_str
chap_num para_num sent_num                                         
2        0        0        0  (Between, IN)   IN   Between  between
                           1      (the, DT)   DT       the      the
                           2   (silver, NN)   NN    silver   silver
                           3   (ribbon, NN)   NN    ribbon   ribbon
                           4       (of, IN)   IN        of       of
...                                     ...  ...       ...      ...
28       390      0        1   (can't, VBP)  VBP     can't    can't
                           2     (stop, VB)   VB      stop     stop
                           3      (for, IN)   IN       for      for
                           4      (the, DT)   DT       the      the
                           5  (inquiry, NN)   NN   inquiry  inquiry

[78996 rows x 4 columns]

In [80]:
df7 = TOKENS
df7['book_id'] = 7
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7], keys=[1,2,3,4,5,6,7], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                      
1       1        0        0        0        (The, DT)   DT        The   
                                   1  (slanting, VBG)  VBG   slanting   
                                   2      (rays, NNS)  NNS       rays   
                                   3         (of, IN)   IN         of   
                                   4  (afternoon, NN)   NN  afternoon   
...                                               ...  ...        ...   
7       28       390      0        1     (can't, VBP)  VBP      can't   
                                   2       (stop, VB)   VB       stop   
                                   3        (for, IN)   IN        for   
                                   4        (the, DT)   DT        the   
                                   5    (inquiry, NN)   NN    inquiry   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
7       28       390      0        1      can't        7  
                                   2       stop        7  
                                   3        for        7  
                                   4        the        7  
                                   5    inquiry        7  

[492742 rows x 5 columns]

### df8

In [81]:
text_file = LIB['source_file_path'].iloc[7]
text_file

'100_english_novels-master/corpus/project/Chesterton_Napoleon_1904.txt'

In [82]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+--", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

The Napoleon of Notting Hill



In [83]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-83-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos token_str term_str
chap_num para_num sent_num                                          
1        1        0        0       (The, DT)   DT       The      the
                           1     (human, JJ)   JJ     human    human
                           2     (race,, NN)   NN     race,    race,
                           3        (to, TO)   TO        to       to
                           4    (which, WDT)  WDT     which    which
...                                      ...  ...       ...      ...
15       190      0        11      (the, DT)   DT       the      the
         191      0        0   (unknown, JJ)   JJ   unknown  unknown
                           1     (world, NN)   NN     world    world
         192      0        0       (THE, DT)   DT       THE      the
                           1       (END, NN)   NN       END      end

[54652 rows x 4 columns]

In [84]:
df8 = TOKENS
df8['book_id'] = 8
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8], keys=[1,2,3,4,5,6,7,8], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
8       15       190      0        11        (the, DT)   DT        the   
                 191      0        0     (unknown, JJ)   JJ    unknown   
                                   1       (world, NN)   NN      world   
                 192      0        0         (THE, DT)   DT        THE   
                                   1         (END, NN)   NN        END   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
8       15       190      0        11        the        8  
                 191      0        0     unknown        8  
                                   1       world        8  
                 192      0        0         the        8  
                                   1         end        8  

[547394 rows x 5 columns]

### df9

In [85]:
text_file = LIB['source_file_path'].iloc[9]
text_file

'100_english_novels-master/corpus/project/Conrad_Nostromo_1904.txt'

In [86]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

NOSTROMO



In [87]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-87-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                               
1        0        0        0     (Sulaco, NNP)  NNP     Sulaco     sulaco
                           1            (-, :)    :          -          -
                           2         (the, DT)   DT        the        the
                           3   (luxuriant, JJ)   JJ  luxuriant  luxuriant
                           4      (beauty, NN)   NN     beauty     beauty
...                                        ...  ...        ...        ...
29       553      0        7  (conquests, NNS)  NNS  conquests  conquests
                           8          (of, IN)   IN         of         of
                           9    (treasure, NN)   NN   treasure   treasure
         554      0        0         (and, CC)   CC        and        and
                           1        (love, VB)   VB       love       love

[169032 rows x 4 columns]

In [88]:
df9 = TOKENS
df9['book_id'] = 9
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9], keys=[1,2,3,4,5,6,7,8,9], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
9       29       553      0        7  (conquests, NNS)  NNS  conquests   
                                   8          (of, IN)   IN         of   
                                   9    (treasure, NN)   NN   treasure   
                 554      0        0         (and, CC)   CC        and   
                                   1        (love, VB)   VB       love   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
9       29       553      0        7  conquests        9  
                                   8         of        9  
                                   9   treasure        9  
                 554      0        0        and        9  
                                   1       love        9  

[716426 rows x 5 columns]

### df10

In [89]:
text_file = LIB['source_file_path'].iloc[8]
text_file

'100_english_novels-master/corpus/project/Conrad_Almayer_1895.txt'

In [90]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+\.", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

ALMAYER'S FOLLY: A STORY OF AN EASTERN RIVER



In [91]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-91-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos      token_str  \
chap_num para_num sent_num                                               
1        0        0        0             (The, DT)   DT            The   
                           1      (well-known, JJ)   JJ     well-known   
                           2          (shrill, NN)   NN         shrill   
                           3           (voice, NN)   NN          voice   
                           4       (startled, VBD)  VBD       startled   
...                                            ...  ...            ...   
12       603      0        5          (Allah, NNP)  NNP          Allah   
                  1        0             (The, DT)   DT            The   
                           1       (Merciful, NNP)  NNP       Merciful   
                  2        0             (The, DT)   DT            The   
                           1  (Compassionate, NNP)  NNP  Compassionate   

                                   term_str  
chap_num para_num sent_num                   
1        0        0        0            the  
                           1     well-known  
                           2         shrill  
                           3          voice  
                           4       startled  
...                                     ...  
12       603      0        5          allah  
                  1        0            the  
                           1       merciful  
                  2        0            the  
                           1  compassionate  

[62976 rows x 4 columns]

In [92]:
df10 = TOKENS
df10['book_id'] = 10
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10], keys=[1,2,3,4,5,6,7,8,9,10], names=['book_id'])
CORPUS

pos_tuple  pos  \
book_id chap_num para_num sent_num                                
1       1        0        0        0             (The, DT)   DT   
                                   1       (slanting, VBG)  VBG   
                                   2           (rays, NNS)  NNS   
                                   3              (of, IN)   IN   
                                   4       (afternoon, NN)   NN   
...                                                    ...  ...   
10      12       603      0        5          (Allah, NNP)  NNP   
                          1        0             (The, DT)   DT   
                                   1       (Merciful, NNP)  NNP   
                          2        0             (The, DT)   DT   
                                   1  (Compassionate, NNP)  NNP   

                                          token_str       term_str  book_id  
book_id chap_num para_num sent_num                                           
1       1        0        0        0            The            the        1  
                                   1       slanting       slanting        1  
                                   2           rays           rays        1  
                                   3             of             of        1  
                                   4      afternoon      afternoon        1  
...                                             ...            ...      ...  
10      12       603      0        5          Allah          allah       10  
                          1        0            The            the       10  
                                   1       Merciful       merciful       10  
                          2        0            The            the       10  
                                   1  Compassionate  compassionate       10  

[779402 rows x 5 columns]

In [93]:
CORPUS.to_csv('corpus6.csv')

### df11

In [98]:
text_file = LIB['source_file_path'].iloc[10]
text_file

'100_english_novels-master/corpus/project/Doyle_Hound_1902.txt'

In [99]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"\sChapter\s[0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

Arthur Conan Doyle



In [100]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-100-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos     token_str  \
chap_num para_num sent_num                                              
1        0        0        0             (The, DT)   DT           The   
                           1          (Curse, NNP)  NNP         Curse   
                           2              (of, IN)   IN            of   
                           3             (the, DT)   DT           the   
                           4   (Baskervilles, NNP)  NNP  Baskervilles   
...                                            ...  ...           ...   
13       27       12       20         (little, JJ)   JJ        little   
                           21         (dinner, NN)   NN        dinner   
                           22             (on, IN)   IN            on   
                           23            (the, DT)   DT           the   
                           24            (way, NN)   NN           way   

                                   term_str  
chap_num para_num sent_num                   
1        0        0        0            the  
                           1          curse  
                           2             of  
                           3            the  
                           4   baskervilles  
...                                     ...  
13       27       12       20        little  
                           21        dinner  
                           22            on  
                           23           the  
                           24           way  

[57317 rows x 4 columns]

In [101]:
df11 = TOKENS
df11['book_id'] = 11
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], keys=[1,2,3,4,5,6,7,8,9,10,11], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
11      13       27       12       20     (little, JJ)   JJ     little   
                                   21     (dinner, NN)   NN     dinner   
                                   22         (on, IN)   IN         on   
                                   23        (the, DT)   DT        the   
                                   24        (way, NN)   NN        way   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
11      13       27       12       20     little       11  
                                   21     dinner       11  
                                   22         on       11  
                                   23        the       11  
                                   24        way       11  

[836719 rows x 5 columns]

### df12

In [102]:
text_file = LIB['source_file_path'].iloc[11]
text_file

'100_english_novels-master/corpus/project/Doyle_Lost_1912.txt'

In [103]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[9].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"\sCHAPTER\s[A-Za-z0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

 The Lost World



In [104]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-104-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos token_str  term_str
chap_num para_num sent_num                                             
1        0        1        0       (There, EX)   EX     There     there
                           1        (Are, NNP)  NNP       Are       are
                           2   (Heroisms, NNP)  NNP  Heroisms  heroisms
                           3        (All, NNP)  NNP       All       all
                           4      (Round, NNP)  NNP     Round     round
...                                        ...  ...       ...       ...
16       560      0        11         (to, TO)   TO        to        to
                           12        (me, PRP)  PRP        me        me
         561      0        0      (across, IN)   IN    across    across
                           1         (the, DT)   DT       the       the
                           2       (table, NN)   NN     table     table

[76276 rows x 4 columns]

In [105]:
df12 = TOKENS
df12['book_id'] = 12
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12], keys=[1,2,3,4,5,6,7,8,9,10,11,12], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
12      16       560      0        11         (to, TO)   TO         to   
                                   12        (me, PRP)  PRP         me   
                 561      0        0      (across, IN)   IN     across   
                                   1         (the, DT)   DT        the   
                                   2       (table, NN)   NN      table   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
12      16       560      0        11         to       12  
                                   12         me       12  
                 561      0        0      across       12  
                                   1         the       12  
                                   2       table       12  

[912995 rows x 5 columns]

### df13

In [106]:
text_file = LIB['source_file_path'].iloc[12]
text_file

'100_english_novels-master/corpus/project/Gaskell_Lovers_1863.txt'

In [107]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

SYLVIA'S LOVERS.



In [108]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-108-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos      token_str  \
chap_num para_num sent_num                                               
1        0        0        0      (MONKSHAVEN, NN)   NN     MONKSHAVEN   
         1        0        0              (On, IN)   IN             On   
                           1             (the, DT)   DT            the   
                           2   (north-eastern, JJ)   JJ  north-eastern   
                           3         (shores, NNS)  NNS         shores   
...                                            ...  ...            ...   
45       99       1        20           (year, NN)   NN           year   
                           21            (ago, RB)   RB            ago   
                  2        0               (', '')   ''              '   
         100      0        0             (THE, DT)   DT            THE   
                           1             (END, NN)   NN            END   

                                    term_str  
chap_num para_num sent_num                    
1        0        0        0      monkshaven  
         1        0        0              on  
                           1             the  
                           2   north-eastern  
                           3          shores  
...                                      ...  
45       99       1        20           year  
                           21            ago  
                  2        0               '  
         100      0        0             the  
                           1             end  

[192497 rows x 4 columns]

In [109]:
df13 = TOKENS
df13['book_id'] = 13
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
13      45       99       1        20       (year, NN)   NN       year   
                                   21        (ago, RB)   RB        ago   
                          2        0           (', '')   ''          '   
                 100      0        0         (THE, DT)   DT        THE   
                                   1         (END, NN)   NN        END   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
13      45       99       1        20       year       13  
                                   21        ago       13  
                          2        0           '       13  
                 100      0        0         the       13  
                                   1         end       13  

[1105492 rows x 5 columns]

### df14

In [110]:
text_file = LIB['source_file_path'].iloc[13]
text_file

'100_english_novels-master/corpus/project/Gaskell_Wives_1865.txt'

In [111]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+\.", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

WIVES AND DAUGHTERS.



In [112]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-112-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple   pos token_str term_str
chap_num para_num sent_num                                            
1        0        0        0        (THE, DT)    DT       THE      the
                           1      (DAWN, NNP)   NNP      DAWN     dawn
                           2         (OF, IN)    IN        OF       of
                           3         (A, NNP)   NNP         A        a
                           4      (GALA, NNP)   NNP      GALA     gala
...                                       ...   ...       ...      ...
60       106      6        17  (Cynthia, NNP)   NNP   Cynthia  cynthia
                           18       (and, CC)    CC       and      and
                           19      (my, PRP$)  PRP$        my       my
                           20       (new, JJ)    JJ       new      new
                           21     (shawl, NN)    NN     shawl    shawl

[269811 rows x 4 columns]

In [113]:
df14 = TOKENS
df14['book_id'] = 14
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14], names=['book_id'])
CORPUS

pos_tuple   pos  token_str  \
book_id chap_num para_num sent_num                                        
1       1        0        0        0         (The, DT)    DT        The   
                                   1   (slanting, VBG)   VBG   slanting   
                                   2       (rays, NNS)   NNS       rays   
                                   3          (of, IN)    IN         of   
                                   4   (afternoon, NN)    NN  afternoon   
...                                                ...   ...        ...   
14      60       106      6        17   (Cynthia, NNP)   NNP    Cynthia   
                                   18        (and, CC)    CC        and   
                                   19       (my, PRP$)  PRP$         my   
                                   20        (new, JJ)    JJ        new   
                                   21      (shawl, NN)    NN      shawl   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
14      60       106      6        17    cynthia       14  
                                   18        and       14  
                                   19         my       14  
                                   20        new       14  
                                   21      shawl       14  

[1375303 rows x 5 columns]

### df15

In [114]:
text_file = LIB['source_file_path'].iloc[14]
text_file

'100_english_novels-master/corpus/project/Lee_Brown_1884.txt'

In [115]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z0-9]+\.", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

Miss Brown



In [116]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-116-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos   token_str    term_str
chap_num para_num sent_num                                                   
1        0        0        0            (IT, NN)   NN          IT          it
         1        0        0          (was, VBD)  VBD         was         was
                           1   (melancholy, VBN)  VBN  melancholy  melancholy
                           2            (to, TO)   TO          to          to
                           3         (admit, VB)   VB       admit       admit
...                                          ...  ...         ...         ...
26       67       1        15        (woman, NN)   NN       woman       woman
                           16      (Walter, NNP)  NNP      Walter      walter
                           17         (has, VBZ)  VBZ         has         has
                           18         (ever, RB)   RB        ever        ever
                           19        (seen, VBN)  VBN        seen        seen

[48576 rows x 4 columns]

In [117]:
df15 = TOKENS
df15['book_id'] = 15
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
15      26       67       1        15      (woman, NN)   NN      woman   
                                   16    (Walter, NNP)  NNP     Walter   
                                   17       (has, VBZ)  VBZ        has   
                                   18       (ever, RB)   RB       ever   
                                   19      (seen, VBN)  VBN       seen   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
15      26       67       1        15      woman       15  
                                   16     walter       15  
                                   17        has       15  
                                   18       ever       15  
                                   19       seen       15  

[1423879 rows x 5 columns]

In [118]:
CORPUS.to_csv('corpus7.csv')

### df16

In [119]:
text_file = LIB['source_file_path'].iloc[15]
text_file

'100_english_novels-master/corpus/project/Lee_Penelope_1903.txt'

In [120]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"I[AIV]", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

PENELOPE BRANDLING


In [121]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-121-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                                
1        0        0        0   (September, NNP)  NNP  September  september
                           1          (29,, CD)   CD        29,        29,
                           2         (1772, CD)   CD       1772       1772
         1        0        0         (This, DT)   DT       This       this
                           1          (is, VBZ)  VBZ         is         is
...                                         ...  ...        ...        ...
4        50       16       50         (the, DT)   DT        the        the
                           51       (white, JJ)   JJ      white      white
                           52     (wailing, NN)   NN    wailing    wailing
                           53   (Northern, NNP)  NNP   Northern   northern
                           54         (sea, NN)   NN        sea        sea

[20138 rows x 4 columns]

In [122]:
df16 = TOKENS
df16['book_id'] = 16
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
16      4        50       16       50        (the, DT)   DT        the   
                                   51      (white, JJ)   JJ      white   
                                   52    (wailing, NN)   NN    wailing   
                                   53  (Northern, NNP)  NNP   Northern   
                                   54        (sea, NN)   NN        sea   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
16      4        50       16       50        the       16  
                                   51      white       16  
                                   52    wailing       16  
                                   53   northern       16  
                                   54        sea       16  

[1444017 rows x 5 columns]

## df 17

In [123]:
text_file = LIB['source_file_path'].iloc[16]
text_file

'100_english_novels-master/corpus/project/Ward_Ashe_1905.txt'

In [124]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"[IVX]+[IVXLCM]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

The Marriage of William Ashe


In [125]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-125-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple   pos token_str term_str
chap_num para_num sent_num                                          
1        0        0        0    (Ashe, NNP)   NNP      Ashe     ashe
                           1    (took, VBD)   VBD      took     took
                           2    (his, PRP$)  PRP$       his      his
                           3    (seat,, NN)    NN     seat,    seat,
                           4   (dined,, NN)    NN    dined,   dined,
...                                     ...   ...       ...      ...
21       137      6        13     (the, DT)    DT       the      the
                           14  (narrow, JJ)    JJ    narrow   narrow
                           15    (room, NN)    NN      room     room
         138      0        0      (THE, DT)    DT       THE      the
                           1      (END, NN)    NN       END      end

[138818 rows x 4 columns]

In [126]:
df17 = TOKENS
df17['book_id'] = 17
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                       
1       1        0        0        0         (The, DT)   DT        The   
                                   1   (slanting, VBG)  VBG   slanting   
                                   2       (rays, NNS)  NNS       rays   
                                   3          (of, IN)   IN         of   
                                   4   (afternoon, NN)   NN  afternoon   
...                                                ...  ...        ...   
17      21       137      6        13        (the, DT)   DT        the   
                                   14     (narrow, JJ)   JJ     narrow   
                                   15       (room, NN)   NN       room   
                 138      0        0         (THE, DT)   DT        THE   
                                   1         (END, NN)   NN        END   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
17      21       137      6        13        the       17  
                                   14     narrow       17  
                                   15       room       17  
                 138      0        0         the       17  
                                   1         end       17  

[1582835 rows x 5 columns]

### df18

In [127]:
text_file = LIB['source_file_path'].iloc[17]
text_file

'100_english_novels-master/corpus/project/Ward_Harvest_1920.txt'

In [128]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"[IVX]+$", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

HARVEST


In [129]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-129-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos  token_str   term_str
chap_num para_num sent_num                                                
1        0        0        0          (Two, CD)   CD        Two        two
                           1          (old, JJ)   JJ        old        old
                           2   (labourers, NNS)  NNS  labourers  labourers
                           3        (came, VBD)  VBD       came       came
                           4          (out, IN)   IN        out        out
...                                         ...  ...        ...        ...
15       115      5        30        (that, DT)   DT       that       that
                           31      (which, WDT)  WDT      which      which
                           32  (destroyed, VBD)  VBD  destroyed  destroyed
                           33     (Rachel, NNP)  NNP     Rachel     rachel
                           34  (Henderson, NNP)  NNP  Henderson  henderson

[75240 rows x 4 columns]

In [130]:
df18 = TOKENS
df18['book_id'] = 18
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                        
1       1        0        0        0          (The, DT)   DT        The   
                                   1    (slanting, VBG)  VBG   slanting   
                                   2        (rays, NNS)  NNS       rays   
                                   3           (of, IN)   IN         of   
                                   4    (afternoon, NN)   NN  afternoon   
...                                                 ...  ...        ...   
18      15       115      5        30        (that, DT)   DT       that   
                                   31      (which, WDT)  WDT      which   
                                   32  (destroyed, VBD)  VBD  destroyed   
                                   33     (Rachel, NNP)  NNP     Rachel   
                                   34  (Henderson, NNP)  NNP  Henderson   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
18      15       115      5        30       that       18  
                                   31      which       18  
                                   32  destroyed       18  
                                   33     rachel       18  
                                   34  henderson       18  

[1658075 rows x 5 columns]

### df19

In [131]:
text_file = LIB['source_file_path'].iloc[18]
text_file

'100_english_novels-master/corpus/project/Woolf_Lighthouse_1927.txt'

In [132]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"[0-9]+", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

To the Lighthouse


In [133]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-133-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple   pos token_str term_str
chap_num para_num sent_num                                           
1        0        1        0      (Yes,, NN)    NN      Yes,     yes,
                           1        (of, IN)    IN        of       of
                           2   (course,, NN)    NN   course,  course,
                           3        (if, IN)    IN        if       if
                           4      (it's, JJ)    JJ      it's     it's
...                                      ...   ...       ...      ...
43       2        14       12     (had, VBN)   VBN       had      had
                           13     (my, PRP$)  PRP$        my       my
                           14   (vision, NN)    NN    vision   vision
         3        0        0       (THE, DT)    DT       THE      the
                           1       (END, NN)    NN       END      end

[68766 rows x 4 columns]

In [134]:
df19 = TOKENS
df19['book_id'] = 19
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], names=['book_id'])
CORPUS

pos_tuple   pos  token_str  \
book_id chap_num para_num sent_num                                        
1       1        0        0        0         (The, DT)    DT        The   
                                   1   (slanting, VBG)   VBG   slanting   
                                   2       (rays, NNS)   NNS       rays   
                                   3          (of, IN)    IN         of   
                                   4   (afternoon, NN)    NN  afternoon   
...                                                ...   ...        ...   
19      43       2        14       12       (had, VBN)   VBN        had   
                                   13       (my, PRP$)  PRP$         my   
                                   14     (vision, NN)    NN     vision   
                 3        0        0         (THE, DT)    DT        THE   
                                   1         (END, NN)    NN        END   

                                        term_str  book_id  
book_id chap_num para_num sent_num                         
1       1        0        0        0         the        1  
                                   1    slanting        1  
                                   2        rays        1  
                                   3          of        1  
                                   4   afternoon        1  
...                                          ...      ...  
19      43       2        14       12        had       19  
                                   13         my       19  
                                   14     vision       19  
                 3        0        0         the       19  
                                   1         end       19  

[1726841 rows x 5 columns]

### df20

In [135]:
text_file = LIB['source_file_path'].iloc[19]
text_file

'100_english_novels-master/corpus/project/Woolf_Night_1919.txt'

In [136]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.strip()
title = LINES.loc[0].line_str
print(title)
chap_lines = LINES.line_str.str.match(r"CHAPTER\s[A-Za-z]", case=True)
chap_nums = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines, 'chap_num'] = chap_nums
LINES.chap_num = LINES.chap_num.ffill()
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int
LINES = LINES.dropna(subset=['chap_num'])
LINES = LINES.loc[~chap_lines] # Remove everything before Chapter 1
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

NIGHT AND DAY


In [137]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str')
CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]
PARAS = CHAPS['chap_str'].str.split(r'\n\n+', expand=True).stack().to_frame('para_str')
PARAS.index.names = OHCO[:2]
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True).str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS = PARAS['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})
SENTS.index.names = OHCO[:3]
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip()
TOKENS = SENTS['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = OHCO[:4]
TOKENS['term_str'] = TOKENS.token_str.str.replace(r"[\W_]+", '', regex=True).str.lower()
TOKENS = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
TOKENS['term_str'] = TOKENS.token_str.str.lower()
TOKENS

<ipython-input-137-5afce1a989ff>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  CHAPS['chap_str'] = CHAPS.chap_str.str.split('\n\n',1).str[1]


pos_tuple  pos    token_str  \
chap_num para_num sent_num                                           
1        0        0        0  (Considering, VBG)  VBG  Considering   
                           1          (that, IN)   IN         that   
                           2           (the, DT)   DT          the   
                           3        (little, JJ)   JJ       little   
                           4         (party, NN)   NN        party   
...                                          ...  ...          ...   
34       42       22       0          (she, PRP)  PRP          she   
                           1     (murmured, VBD)  VBD     murmured   
                           2          (back, RB)   RB         back   
                           3            (to, TO)   TO           to   
                           4          (him, PRP)  PRP          him   

                                 term_str  
chap_num para_num sent_num                 
1        0        0        0  considering  
                           1         that  
                           2          the  
                           3       little  
                           4        party  
...                                   ...  
34       42       22       0          she  
                           1     murmured  
                           2         back  
                           3           to  
                           4          him  

[159628 rows x 4 columns]

In [138]:
df20 = TOKENS
df20['book_id'] = 20
CORPUS = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20], keys=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], names=['book_id'])
CORPUS

pos_tuple  pos  token_str  \
book_id chap_num para_num sent_num                                      
1       1        0        0        0        (The, DT)   DT        The   
                                   1  (slanting, VBG)  VBG   slanting   
                                   2      (rays, NNS)  NNS       rays   
                                   3         (of, IN)   IN         of   
                                   4  (afternoon, NN)   NN  afternoon   
...                                               ...  ...        ...   
20      34       42       22       0       (she, PRP)  PRP        she   
                                   1  (murmured, VBD)  VBD   murmured   
                                   2       (back, RB)   RB       back   
                                   3         (to, TO)   TO         to   
                                   4       (him, PRP)  PRP        him   

                                       term_str  book_id  
book_id chap_num para_num sent_num                        
1       1        0        0        0        the        1  
                                   1   slanting        1  
                                   2       rays        1  
                                   3         of        1  
                                   4  afternoon        1  
...                                         ...      ...  
20      34       42       22       0        she       20  
                                   1   murmured       20  
                                   2       back       20  
                                   3         to       20  
                                   4        him       20  

[1886469 rows x 5 columns]

In [139]:
CORPUS.to_csv('FINAL.csv')

In [141]:
OHCO

['chap_num', 'para_num', 'sent_num', 'token_num']

In [147]:
OHCO2 = ['book_id'] + OHCO[:-1]
final_corpus = pd.read_csv('FINAL.csv').set_index(OHCO2)

In [149]:
final_corpus.columns

Index(['Unnamed: 4', 'pos_tuple', 'pos', 'token_str', 'term_str', 'book_id.1'], dtype='object')

In [150]:
final_corpus = final_corpus.rename({'Unnamed: 4':'token_num'},axis=1)
final_corpus

token_num            pos_tuple  pos  \
book_id chap_num para_num sent_num                                        
1       1        0        0                 0        ('The', 'DT')   DT   
                          0                 1  ('slanting', 'VBG')  VBG   
                          0                 2      ('rays', 'NNS')  NNS   
                          0                 3         ('of', 'IN')   IN   
                          0                 4  ('afternoon', 'NN')   NN   
...                                       ...                  ...  ...   
20      34       42       22                0       ('she', 'PRP')  PRP   
                          22                1  ('murmured', 'VBD')  VBD   
                          22                2       ('back', 'RB')   RB   
                          22                3         ('to', 'TO')   TO   
                          22                4       ('him', 'PRP')  PRP   

                                    token_str   term_str  book_id.1  
book_id chap_num para_num sent_num                                   
1       1        0        0               The        the          1  
                          0          slanting   slanting          1  
                          0              rays       rays          1  
                          0                of         of          1  
                          0         afternoon  afternoon          1  
...                                       ...        ...        ...  
20      34       42       22              she        she         20  
                          22         murmured   murmured         20  
                          22             back       back         20  
                          22               to         to         20  
                          22              him        him         20  

[1886469 rows x 6 columns]

In [151]:
final_corpus.to_csv('corpusFINAL.csv')

### Make metadata (LIB) file

In [446]:
LIB

,source_file_path,raw_title,regex
book_id,,,
1,100_english_novels-master/corpus/project/Barcl...,Ladies,'CHAPTER\\s[A-Za-z]'
2,100_english_novels-master/corpus/project/Barcl...,Postern,'^THE\\s[A-Za-z]+ DAY$'
3,100_english_novels-master/corpus/project/Benne...,Babylon,'^Chapter\\s'
4,100_english_novels-master/corpus/project/Benne...,Helen,'^Chapter\\s[0-9]+'
5,100_english_novels-master/corpus/project/Burne...,Garden,'^CHAPTER\\s[A-Za-z]'
6,100_english_novels-master/corpus/project/Burne...,Princess,'^\\d\\s'
7,100_english_novels-master/corpus/project/Chest...,Innocence,'^\\s[A-Za-z]+ '
8,100_english_novels-master/corpus/project/Chest...,Napoleon,'^CHAPTER\\s[A-Za-z]'
9,100_english_novels-master/corpus/project/Conra...,Almayer,''


In [447]:
authors = ['Florence L. Barclay','Florence L. Barclay','Arnold Bennett','Arnold Bennett','Frances Hodgson Burnett'
           ,'Frances Hodgson Burnett','G. K. Chesterton','G. K. Chesterton','Joseph Conrad','Joseph Conrad',
          'Arthur Conan Doyle','Arthur Conan Doyle','ELIZABETH GASKELL','ELIZABETH GASKELL','Vernon Lee',
          'Vernon Lee','Mary Augusta Ward','Mary Augusta Ward','Virginia Woolf','Virginia Woolf']
gender = ['female','female','male','male','male','male','male','male','male','male','male','male',
         'female','female','female','female','female','female','female','female']

In [448]:
LIB['author'] = authors
LIB['gender'] = gender

In [450]:
LIB

,source_file_path,raw_title,regex,author,gender
book_id,,,,,
1,100_english_novels-master/corpus/project/Barcl...,Ladies,'CHAPTER\\s[A-Za-z]',Florence L. Barclay,female
2,100_english_novels-master/corpus/project/Barcl...,Postern,'^THE\\s[A-Za-z]+ DAY$',Florence L. Barclay,female
3,100_english_novels-master/corpus/project/Benne...,Babylon,'^Chapter\\s',Arnold Bennett,male
4,100_english_novels-master/corpus/project/Benne...,Helen,'^Chapter\\s[0-9]+',Arnold Bennett,male
5,100_english_novels-master/corpus/project/Burne...,Garden,'^CHAPTER\\s[A-Za-z]',Frances Hodgson Burnett,male
6,100_english_novels-master/corpus/project/Burne...,Princess,'^\\d\\s',Frances Hodgson Burnett,male
7,100_english_novels-master/corpus/project/Chest...,Innocence,'^\\s[A-Za-z]+ ',G. K. Chesterton,male
8,100_english_novels-master/corpus/project/Chest...,Napoleon,'^CHAPTER\\s[A-Za-z]',G. K. Chesterton,male
9,100_english_novels-master/corpus/project/Conra...,Almayer,'',Joseph Conrad,male
